In [8]:
import pandas as pd
from pathlib import Path

p = Path(".")
files = list(p.resolve().rglob("0*.jsonl"))
print(files)

df = pd.concat([pd.read_json(f, orient="records", lines=True) for f in files], ignore_index=True)



from sklearn.metrics import accuracy_score, f1_score

df["accuracy"] = df.apply(
    lambda row: accuracy_score(row["y_true"], row["y_pred"]), axis=1
)
df["macroF1"] = df.apply(
    lambda row: f1_score(row["y_true"], row["y_pred"], average="macro"), axis=1
)

def assign_model(row):
    try:
        return row["train_config"]["model_name_or_path"]
    except:
        return "/".join(row["model_name_or_path"].split("/")[-2].split("_")[0:2])

df["model"] = df.apply(assign_model, axis=1)

df.drop_duplicates(subset=["accuracy", "model_name_or_path",'eval_file', "clip_seconds"],inplace=True)
c1 = df.output_column == "Speaker_name"
c2 = df.clip_seconds == 2
c3 = [True if 0.8 < i < 1.1 else False for i in df.accuracy.values]
c4 = df.eval_file.str.contains("test")
df.loc[c1&c2&c4, ["model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']].sort_values(by="accuracy", ascending=False)

[PosixPath('/home/peterr/macocu/task11/010_results.jsonl'), PosixPath('/home/peterr/macocu/task11/011_results.jsonl'), PosixPath('/home/peterr/macocu/task11/013_results_english.jsonl'), PosixPath('/home/peterr/macocu/task11/008_results.jsonl'), PosixPath('/home/peterr/macocu/task11/012_results_nonslavic.jsonl'), PosixPath('/home/peterr/macocu/task11/012_results.jsonl'), PosixPath('/home/peterr/macocu/task11/013_results_slavic_asr.jsonl')]


,model_name_or_path,clip_seconds,eval_file,accuracy,macroF1,model
45,/home/peterr/macocu/task11/models/classla_wav2...,2,003_speaker_id_test_for_datasets.csv,1.000,1.000000,classla/wav2vec2-large-slavic-parlaspeech-hr
26,models/facebook_wav2vec2-large-slavic-voxpopul...,2,003_speaker_id_test_for_datasets.csv,0.806,0.784407,facebook/wav2vec2-large-slavic-voxpopuli-v2
5,models/facebook_wav2vec2-large-960h-lv60-self_...,2,003_speaker_id_test_for_datasets.csv,0.106,0.047947,facebook/wav2vec2-large-960h-lv60-self


In [10]:
c1 = df.output_column == "Speaker_name"
c2 = df.clip_seconds == -1
c3 = [True if 0.8 < i < 1.1 else False for i in df.accuracy.values]
c4 = df.eval_file.str.contains("test")
df.loc[c1&c2&c4, ["model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']].sort_values(by="accuracy", ascending=False)

,model_name_or_path,clip_seconds,eval_file,accuracy,macroF1,model
43,/home/peterr/macocu/task11/models/classla_wav2...,-1,003_speaker_id_test_for_datasets.csv,1.000,1.000000,classla/wav2vec2-large-slavic-parlaspeech-hr
28,models/facebook_wav2vec2-large-slavic-voxpopul...,-1,003_speaker_id_test_for_datasets.csv,0.998,0.997995,facebook/wav2vec2-large-slavic-voxpopuli-v2
7,models/facebook_wav2vec2-large-960h-lv60-self_...,-1,003_speaker_id_test_for_datasets.csv,0.334,0.274969,facebook/wav2vec2-large-960h-lv60-self


In [12]:
c1 = df.output_column == "Speaker_gender"

c4 = df.eval_file.str.contains("test")
subset = df.loc[c1&c4, ["model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']]#.sort_values(by="accuracy", ascending=False)
subset.groupby("model_name_or_path clip_seconds".split()).agg({"accuracy": "mean", "macroF1": "mean"})

,,accuracy,macroF1
model_name_or_path,clip_seconds,,
/home/peterr/macocu/task11/models/classla_wav2vec2-large-slavic-parlaspeech-hr_NEW_GENDER_/checkpoint-250,-1,0.9850,0.984997
/home/peterr/macocu/task11/models/classla_wav2vec2-large-slavic-parlaspeech-hr_NEW_GENDER_2s_/checkpoint-250,2,0.9850,0.984997
models/facebook_wav2vec2-large-960h-lv60-self_NEW_GENDER_/checkpoint-250,-1,0.9990,0.999000
models/facebook_wav2vec2-large-960h-lv60-self_NEW_GENDER_2s_/checkpoint-250,2,0.9935,0.993500
models/facebook_wav2vec2-large-slavic-voxpopuli-v2_NEW_GENDER_/checkpoint-250,-1,0.9970,0.997000
models/facebook_wav2vec2-large-slavic-voxpopuli-v2_NEW_GENDER_2s_/checkpoint-250,2,0.9895,0.989499


In [13]:
c1 = df.output_column == "Speaker_age_group"

c4 = df.eval_file.str.contains("test")
subset = df.loc[c1&c4, ["model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']]#.sort_values(by="accuracy", ascending=False)
subset.groupby("model_name_or_path clip_seconds".split()).agg({"accuracy": "mean", "macroF1": "mean"})

,,accuracy,macroF1
model_name_or_path,clip_seconds,,
/home/peterr/macocu/task11/models/classla_wav2vec2-large-slavic-parlaspeech-hr_NEW_AGE_ID_/checkpoint-2250,-1,0.722,0.721715
models/facebook_wav2vec2-large-960h-lv60-self_NEW_AGE_ID_/checkpoint-2250,-1,0.678,0.672112
models/facebook_wav2vec2-large-slavic-voxpopuli-v2_NEW_AGE_ID_/checkpoint-2250,-1,0.694,0.689971


In [14]:
c1 = df.output_column == "Party_status"

c4 = df.eval_file.str.contains("test")
subset = df.loc[c1&c4, ["model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']]#.sort_values(by="accuracy", ascending=False)
subset.groupby("model_name_or_path clip_seconds".split()).agg({"accuracy": "mean", "macroF1": "mean"})

,,accuracy,macroF1
model_name_or_path,clip_seconds,,
/home/peterr/macocu/task11/models/facebook_wav2vec2-large-960h-lv60-self_PARTY_STATUS_/checkpoint-4680,-1,0.548889,0.531373
/home/peterr/macocu/task11/models/facebook_wav2vec2-large-slavic-voxpopuli-v2_PARTY_STATUS_/checkpoint-4680,-1,0.590000,0.587285
models/classla_wav2vec2-large-slavic-parlaspeech-hr_PARTY_STATUS_/checkpoint-4680,-1,0.626667,0.625928


In [16]:
c1 = df.model_name_or_path.str.contains('facebook_wav2vec2-large-slavic-voxpopuli-')

c4 = df.eval_file.str.contains("dev")
subset = df.loc[c1&c4, ["output_column", "model_name_or_path", "clip_seconds", "eval_file", "accuracy", "macroF1", 'model']]#.sort_values(by="accuracy", ascending=False)
subset.groupby("output_column clip_seconds".split()).agg({"accuracy": "mean", "macroF1": "mean"})

accuracy   macroF1
output_column     clip_seconds                    
Party_status      -1            0.613333  0.597222
Speaker_age_group -1            0.704000  0.695191
Speaker_gender    -1            0.994000  0.994000
                   2            1.000000  1.000000
Speaker_name      -1            0.992000  0.991667
                   2            0.738000  0.703877